### Libraries

In [ ]:
import networkx as nx
from shapely.geometry import Point,LineString,Polygon

import simpy
import xarray as xr

from opentnsim import core as core_module
from opentnsim import graph as graph_module
from opentnsim import lock_new as lock_module
from opentnsim import energy as energy_module
from opentnsim import output as output_module
from opentnsim import vessel as vessel_module
from opentnsim import vessel_traffic_service as vessel_traffic_service_module
from opentnsim import zeesluisformulering as ZSF
from netCDF4 import Dataset
import datetime
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
import folium
from shapely.ops import nearest_points, linemerge, split, snap, transform
import pyproj
import re
path = os.getcwd()

### Simulation

In [ ]:
env = simpy.Environment()
t_start = datetime.datetime(2025, 1, 1, 0, 0, 0)
t_stop = datetime.datetime(2026, 1, 1, 0, 0, 0)

env = simpy.Environment(initial_time=t_start.timestamp())
env.epoch = t_start
env.simulation_start = t_start
env.simulation_stop = t_stop

### Network

In [ ]:
wgs84eqd = pyproj.CRS('4087')
wgs84rad = pyproj.CRS('4326')
wgs84eqd_to_wgs84rad = pyproj.transformer.Transformer.from_crs(wgs84eqd,wgs84rad,always_xy=True).transform
wgs84rad_to_wgs84eqd = pyproj.transformer.Transformer.from_crs(wgs84rad,wgs84eqd,always_xy=True).transform

graph = nx.MultiDiGraph()
graph.add_node('A',geometry=transform(wgs84eqd_to_wgs84rad,Point(-5000,0)))
graph.add_node('B',geometry=transform(wgs84eqd_to_wgs84rad,Point(5000,0)))
graph.add_edge('A','B',geometry=transform(wgs84eqd_to_wgs84rad,LineString([Point(-5000,0),Point(5000,0)])),length=10000)
graph.add_edge('B','A',geometry=transform(wgs84eqd_to_wgs84rad,LineString([Point(5000,0),Point(-5000,0)])),length=10000)
pos = {edge:(point.x,point.y) for edge,point in nx.get_node_attributes(graph, 'geometry').items()}
nx.draw(graph,pos,with_labels=True)
env.FG = graph

### Hydrodynamics

In [ ]:
if not os.path.isdir(path+"\\hydrodynamic_data\\"):
    os.mkdir(path+"\\hydrodynamic_data\\")
if "hydrodynamic_data_test.nc" not in os.listdir(path+"\\hydrodynamic_data\\"):
    hydrodynamic_data = xr.Dataset()
    times = np.arange(pd.Timestamp('2025-01-01 00:00'),pd.Timestamp('2026-01-01 00:00'),pd.Timedelta(5,'min'))
    nodes = np.array(['A','B'])
    wlevs = np.array([np.array([np.sin((2*np.pi/(12.5*60*60))*x) for x in np.arange(0,365*24*60*60,5*60)]),np.zeros(len(times))])
    salinity = np.array([30*np.ones(len(times)),np.zeros(len(times))])
    wlev_data = xr.DataArray(data=wlevs,coords={'STATION':nodes,'TIME':times})
    salinity_data = xr.DataArray(data=salinity,coords={'STATION':nodes,'TIME':times})
    hydrodynamic_data['Water level'] = wlev_data
    hydrodynamic_data['Salinity'] = salinity_data
    hydrodynamic_data.to_netcdf(path+"\\hydrodynamic_data\\hydrodynamic_data_test.nc")

### Vessel speed

In [ ]:
if not os.path.isdir(path+"\\speed_data\\"):
    os.mkdir(path+"\\speed_data\\")
if "speed_data_test.pkl" not in os.listdir(path+"\\speed_data\\"):
    edge_index = [('A','B',0),('B','A',0)]
    data = np.array([6,6])
    vessel_speed_data = pd.DataFrame(data=data,columns=['Speed'],index=edge_index)
    vessel_speed_data.to_pickle(path + '\\speed_data\\speed_data_test.pkl')

### Vessel traffic services

In [ ]:
env.vessel_traffic_service = vessel_traffic_service_module.VesselTrafficService(FG=graph, 
                                                                                hydrodynamic_start_time = pd.Timestamp('2025-01-01 00:00'),
                                                                                hydrodynamic_information_path = path+"\\hydrodynamic_data\\hydrodynamic_data_test.nc", 
                                                                                vessel_speed_data_path = path + '\\speed_data\\speed_data_test.pkl')

### Lock object

In [ ]:
lock = lock_module.IsLockComplex(env=env,
                                 name='Lock',
                                 start_node='A',
                                 node_open='A',
                                 end_node='B',
                                 approach_node_A = 'A',
                                 approach_node_B = 'B',
                                 distance_lock_doors_A_to_waiting_area_A = 4800,
                                 distance_lock_doors_B_to_waiting_area_B = 4800,
                                 distance_from_start_node_to_lock_doors_A = 4800,
                                 distance_from_end_node_to_lock_doors_B = 4800,
                                 lock_length = 400,
                                 lock_width = 50,
                                 lock_depth = 15,
                                 levelling_time = 300,
                                 disch_coeff = 0.65,
                                 gate_opening_time = 120,
                                 opening_area = 80,
                                 sailing_distance_to_crossing_point = 1800,
                                 doors_opening_time= 300,
                                 doors_closing_time= 300,
                                 sailing_in_time_gap = 300,
                                 sailing_in_speed = 0.5,
                                 sailing_out_time_gap = 120,
                                 time_step = 30,
                                 sailing_time_before_opening_lock_doors = 600,
                                 sailing_time_before_closing_lock_doors = 120,
                                 detector_nodes = ['A','B'],
                                 predictive=False)

### Vessels

In [ ]:
Vessel = type('Vessel',(energy_module.ConsumesEnergy,lock_module.PassesLockComplex,vessel_module.IsVessel,),{}) 

In [ ]:
vessel_1 = Vessel(**{"env": env,
                     "name": 'Vessel 1',
                     "origin": 'A',
                     "destination": 'B',
                     "type": 'tanker',
                     "bound": 'outbound',
                     "L": 100,
                     "B": 20,
                     "T": 10,
                     "v": 4,
                     "P_installed": 50000*1000,
                     "C_year": 2010,
                     "L_w":3,
                     "arrival_time": pd.Timestamp('2025-01-01 00:00:00')})
vessel_1.process = env.process(vessel_1.move())

vessel_2 = Vessel(**{"env": env,
                     "name": 'Vessel 2',
                     "origin": 'B',
                     "destination": 'A',
                     "type": 'tanker',
                     "bound": 'outbound',
                     "L": 100,
                     "B": 20,
                     "T": 10,
                     "v": 4,
                     "P_installed": 50000*1000,
                     "C_year": 2010,
                     "L_w":3,
                     "C_year": 2010,
                     "arrival_time": pd.Timestamp('2025-01-01 00:05:00')})
vessel_2.process = env.process(vessel_2.move())

vessels = []
vessels.append(vessel_1)
vessels.append(vessel_2)

### Run model

In [ ]:
env.run()

In [ ]:
pd.Timestamp('2025-01-01 00:35:40')+pd.Timedelta(seconds=500)

In [ ]:
lock_df = pd.DataFrame(lock.logbook)
vessel_df1 = pd.DataFrame(vessel_1.logbook)
vessel_df2 = pd.DataFrame(vessel_2.logbook)

In [ ]:
#Testing time that vessel enters the lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_entry_start.round('s').to_pydatetime().timestamp()/10,
                               vessel_df1.loc[2].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

#Testing time that vessel stops in the lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_entry_stop.round('s').to_pydatetime().timestamp()/10,
                               vessel_df1.loc[4].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is closing start
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_door_closing_start.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[0].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is closing stop
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_door_closing_stop.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[1].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock is levelling start
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_levelling_start.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[2].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock is levelling stop
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_levelling_stop.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[3].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is opening start
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_door_opening_start.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[4].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is closing stop
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_door_opening_stop.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[5].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that vessel is starting to sail out of lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_departure_start.round('s').to_pydatetime().timestamp()/10,
                               vessel_df1.loc[7].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that vessel is has sailed out of lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[0].time_departure_stop.round('s').to_pydatetime().timestamp()/10,
                               vessel_df1.loc[8].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

#Testing time that vessel enters the lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_entry_start.round('s').to_pydatetime().timestamp()/10,
                               vessel_df2.loc[5].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

#Testing time that vessel stops in the lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_entry_stop.round('s').to_pydatetime().timestamp()/10,
                               vessel_df2.loc[6].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is closing start
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_door_closing_start.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[6].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is closing stop
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_door_closing_stop.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[7].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock is levelling start
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_levelling_start.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[8].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock is levelling stop
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_levelling_stop.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[9].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is opening start
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_door_opening_start.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[10].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that lock door is closing stop
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_door_opening_stop.round('s').to_pydatetime().timestamp()/10,
                               lock_df.loc[11].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that vessel is starting to sail out of lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_departure_start.round('s').to_pydatetime().timestamp()/10,
                               vessel_df2.loc[9].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

# Testing time that vessel is has sailed out of lock
np.testing.assert_almost_equal(lock.operation_planning.iloc[1].time_departure_stop.round('s').to_pydatetime().timestamp()/10,
                               vessel_df2.loc[10].Timestamp.to_pydatetime().timestamp()/10,decimal=0)

np.testing.assert_equal(len(lock_df),12)
print('test check')